In [1]:
import mummichog as mg
import pandas as pd
import numpy as np
import os
import re

# run_name = "1734535932.3620632.ttest_p0_05"
run_name = "trans_omic_covid_data.run_1_default"
end_name = run_name.split(".")[-1]

In [2]:
## MUMMICHOG OUTPUT: COMPARING NAME OF METABOLITE IN GT DATASET TO MUMMICHOG ANNOTATION ##
print("Mummichog Output - User input to emperical compounds\n")

file_path = f"../runs/{run_name}/tables/userInput_to_EmpiricalCompounds.tsv"
og_ui_cpd = pd.read_csv(file_path, sep="\t")
print(f"Number of original ouput rows: {len(og_ui_cpd)}")
ui_cpd_nan = og_ui_cpd.dropna(subset=['compound_names']).copy()
print(f"Dropped {len(og_ui_cpd)-len(ui_cpd_nan)} nan compound_names rows, {len(ui_cpd_nan)} rows")
print(f"Number of unique features: {len(np.unique(ui_cpd_nan['CompoundID_from_user']))}")

# a metabolite feature can have multiple annotations separated by a "$"
ui_cpd_nan['compound_names_split'] = ui_cpd_nan['compound_names'].str.split('$')
ui_cpd = ui_cpd_nan.explode('compound_names_split')
ui_cpd = ui_cpd.rename(columns={'compound_names_split': 'compound_name'})
ui_cpd = ui_cpd.drop(columns=['compound_names'])
ui_cpd = ui_cpd[ui_cpd['compound_name'].str.strip() != ""]
print(f"Number of rows after $ split: {len(ui_cpd)}")


# ui_cpd['compound_name_no_colon'] = ui_cpd['compound_name'].str.split(';').str[0]
# ui_cpd.to_csv(f"../runs/{run_name}/tables/edited_emperical_compounds_no_symbols.csv")
ui_cpd = ui_cpd.reset_index(drop=True)
ui_cpd.head()


Mummichog Output - User input to emperical compounds

Number of original ouput rows: 291
Dropped 16 nan compound_names rows, 275 rows
Number of unique features: 262
Number of rows after $ split: 449


,input_row,EID,str_row_ion,compounds,input_row.1,m/z,retention_time,p_value,statistic,CompoundID_from_user,compound_name
0,row1,E1,row1_M+H[1+],CE1944,row1,70.065243,44.397226,0.153276,-2.008318,M70.065243T44.3972,1-pyrroline
1,row14,E4,row14_M+H[1+],C01659,row14,72.044165,582.519073,0.006767,3.525478,M72.044165T582.5191,Acrylamide; 2-Propenamide
2,row33,E5,row33_M+H[1+];row675_M+HCOONa[1+],C01888;C05665,row33,74.060176,43.183455,0.775557,0.504408,M74.060176T43.1835,Aminoacetone
3,row33,E5,row33_M+H[1+];row675_M+HCOONa[1+],C01888;C05665,row33,74.060176,43.183455,0.775557,0.504408,M74.060176T43.1835,beta-Aminopropion aldehyde
4,row33,E12,row33_M+2H[2+];row736_M+H[1+];row750_M(C13)+H[...,C00047,row33,74.060176,43.183455,0.775557,0.504408,M74.060176T43.1835,L-Lysine


In [3]:
# Adding KEGG to ui_cpd df
KEGG_map_file = "name_map_on_mummichog_output_cpds.csv"
KEGG_map = pd.read_csv(KEGG_map_file)
KEGG_map['compound_name'] = KEGG_map['Query']
KEGG_map = KEGG_map.drop(columns=['Query'])

ui_cpd = ui_cpd.merge(KEGG_map[['compound_name', 'KEGG']], on='compound_name', how='left')
# print(len(ui_cpd))
# ui_cpd.to_csv(f"../runs/{run_name}/tables/edited_emperical_compounds_KEGG.csv")
ui_cpd.head()

,input_row,EID,str_row_ion,compounds,input_row.1,m/z,retention_time,p_value,statistic,CompoundID_from_user,compound_name,KEGG
0,row1,E1,row1_M+H[1+],CE1944,row1,70.065243,44.397226,0.153276,-2.008318,M70.065243T44.3972,1-pyrroline,C15668
1,row14,E4,row14_M+H[1+],C01659,row14,72.044165,582.519073,0.006767,3.525478,M72.044165T582.5191,Acrylamide; 2-Propenamide,NaN
2,row33,E5,row33_M+H[1+];row675_M+HCOONa[1+],C01888;C05665,row33,74.060176,43.183455,0.775557,0.504408,M74.060176T43.1835,Aminoacetone,C01888
3,row33,E5,row33_M+H[1+];row675_M+HCOONa[1+],C01888;C05665,row33,74.060176,43.183455,0.775557,0.504408,M74.060176T43.1835,beta-Aminopropion aldehyde,C05665
4,row33,E12,row33_M+2H[2+];row736_M+H[1+];row750_M(C13)+H[...,C00047,row33,74.060176,43.183455,0.775557,0.504408,M74.060176T43.1835,L-Lysine,C00047


In [4]:
# True metabolite identities from the COVID-dataset (gt -> ground truth)
# Required cols: metabolite_identification, retention_time
file_path = "m_MTBLS2542_Metabo_LC-MS_positive_reverse-phase_v2_maf.tsv"
gt = pd.read_csv(file_path, sep="\t")
gt['retention_time'] = gt['retention_time']*60

print("Ground Truth - True Metabolite info from COVID dataset")
print(f"No of rows: {len(gt)}")
print(f"No of unique KEGG IDs {len(np.unique(gt['database_identifier']))}")
gt.head()

Ground Truth - True Metabolite info from COVID dataset
No of rows: 515
No of unique KEGG IDs 503


,database_identifier,chemical_formula,smiles,inchi,metabolite_identification,mass_to_charge,fragmentation,modifications,charge,retention_time,...,20P02820143_BEHC18_POS,20P02820140_BEHC18_POS,20P02820141_BEHC18_POS,20P02820209_BEHC18_POS,20P02820208_BEHC18_POS,20P02820075_BEHC18_POS,20P02820073_BEHC18_POS,20P02820072_BEHC18_POS,20P02820079_BEHC18_POS,20P02820078_BEHC18_POS
0,CHEBI:16610,C7H19N3,NCCCCNCCCN,"InChI=1S/C7H19N3/c8-4-1-2-6-10-7-3-5-9/h10H,1-9H2",Spermidine,NaN,NaN,NaN,NaN,36.78,...,7.528005e+05,2.562570e+06,2.398701e+06,4.761392e+06,1.558479e+06,1.824217e+06,9.877502e+05,1.373717e+06,2.332980e+06,1.791080e+06
1,CHEBI:18019,C6H14N2O2,NCCCC[C@H](N)C(O)=O,"InChI=1S/C6H14N2O2/c7-4-2-1-3-5(8)6(9)10/h5H,1...",L-lysine,NaN,NaN,NaN,NaN,36.78,...,2.070799e+08,2.697232e+08,2.632105e+08,1.911982e+08,1.692379e+08,2.475934e+08,2.119148e+08,1.757703e+08,1.667598e+08,2.833034e+08
2,CHEBI:17964,C6H11NO2,OC(=O)C1CCCCN1,"InChI=1S/C6H11NO2/c8-6(9)5-3-1-2-4-7-5/h5,7H,1...",Pipecolate,NaN,NaN,NaN,NaN,36.84,...,2.020358e+08,2.628740e+08,2.573673e+08,1.866568e+08,1.652232e+08,2.419402e+08,2.069042e+08,1.713392e+08,1.631715e+08,2.777294e+08
3,CHEBI:16176,C5H12N2O2,NCCC[C@@H](N)C(O)=O,"InChI=1S/C5H12N2O2/c6-3-1-2-4(7)5(8)9/h4H,1-3,...",D-ornithine,NaN,NaN,NaN,NaN,36.84,...,5.277620e+07,6.608178e+07,6.886163e+07,8.334655e+07,1.189747e+08,3.495716e+07,3.587961e+07,4.777418e+07,2.996990e+07,5.202093e+07
4,CHEBI:17311,C9H21N2O2+,OC(=O)[C@@H](N)CCCC[N+](C)(C)C,"InChI=1S/C9H20N2O2/c1-11(2,3)7-5-4-6-8(10)9(12...","Nepsilon,nepsilon,nepsilon-trimethyllysine",NaN,NaN,NaN,NaN,40.92,...,1.798543e+07,1.915268e+07,1.417020e+07,1.017568e+07,4.711219e+06,5.736316e+07,2.130167e+07,2.380881e+07,1.324886e+07,1.216843e+07


In [5]:
def check_cpd_synonyms(compound_name, index):
    ''' Function checks if all compounds in the gt dataset are unique i.e., that there are no synonyms for a compound
    listed as separate entries. Note that this is not exhaustive as this function does not check all compounds in gt but
    only the ones checks the compound synonyms output by mummichog's annotations'''
    
    compound_syns = compound_name.split(";")
    if len(compound_syns) <= 1:
        return True
    else:
        found_match = False
        for compound in compound_syns:
            escaped_compound_name = re.escape(compound)
            match = gt[gt['metabolite_identification'].str.fullmatch(escaped_compound_name, case=False, na=False)]
            if not found_match and not match.empty:
                found_match = True
            else:
                if not match.empty:
                    print("oh no the same compound with a different name exists in more than one ground truth entry", index, compound_name)
                    return False
                
    return True

def check_duplicate_features(matches_df):
    if len(matches_df) == len(np.unique(matches_df["compoundID_from_user"])):
        return matches_df
    else:
        print("The same compound id feature is pointing to different compounds. Fix it")
        grouped = matches_df.groupby("compoundID_from_user")
        dup_mets_df = grouped.filter(lambda x: len(x) > 1)
        # print(dup_mets_df.groupby("compoundID_from_user").head())
        matches_df = matches_df.drop(dup_mets_df.index)

        # The fix: checks exact names; if names are the same, selects the one with the smallest rt_diff
        keep_dup_idx = []
        for compoundID_from_user, grp in dup_mets_df.groupby("compoundID_from_user"):
            keep_index = None # or (index, rt_diff)
            for index, row in grp.iterrows():

                check_names = row['compound_name'].strip().lower() == row['gt_compound_name'].strip().lower()
                
                # exact matching name exists
                if check_names:
                    # multiple rows have the exact name -> pick the smallest rt diff
                    if keep_index:
                        if row['rt_difference'] < keep_index[-1]:
                            keep_index = (index, row['rt_difference'])
                    else:
                        keep_index = (index, row['rt_difference'])
            
            if not keep_index: # i.e. compound_name and gt_compound_name are not exact matches
                print('um', row['compound_name'], row['gt_compound_name'])
                keep_index = (grp['rt_difference'].idxmin(),)

            keep_dup_idx.append(keep_index[0])
            
        fixed_dup_mets_df = dup_mets_df.loc[keep_dup_idx]

        matches_df = pd.concat([matches_df, fixed_dup_mets_df])
        if len(matches_df) == len(np.unique(matches_df["compoundID_from_user"])):
            print(f"All fixed. {len(dup_mets_df)} duplicate compoundID_from_user fixed to {len(fixed_dup_mets_df)}\n")
            print(fixed_dup_mets_df)

        return matches_df

In [6]:
def run_rt_name_matches():
    matches = []

    for index, row in ui_cpd.iterrows():
        compound_match_found = False
        compound_name = row['compound_name']

        if pd.notna(compound_name):     
            ui_retention_time = row['retention_time']   

            # check_cpd_synonyms(compound_name, index)
            compound_syns = compound_name.split(";")
            
            for compound in compound_syns:
                escaped_compound_name = re.escape(compound_name)
                match = gt[gt['metabolite_identification'].str.fullmatch(escaped_compound_name, case=False, na=False)]
                if not match.empty:
                    break
            
            # Include partial matches
            if match.empty:
                match = gt[gt['metabolite_identification'].str.contains(escaped_compound_name, case=False, na=False)]
            
            if not match.empty:
                compound_match_found = True  
                rt_difference, gt_retention_time, gt_compound_name = min(
                    ((abs(ui_retention_time - match.iloc[i]['retention_time']), match.iloc[i]['retention_time'], match.iloc[i]['metabolite_identification']) 
                    for i in range(len(match))),
                    key=lambda x: x[0]
                )

                matches.append({
                    "compound_name": compound_name,
                    "gt_compound_name": gt_compound_name,
                    "ui_retention_time": ui_retention_time,
                    "gt_retention_time": gt_retention_time,
                    "rt_difference": rt_difference,
                    "p_value": row['p_value'],
                    "compoundID_from_user": row['CompoundID_from_user'],
                    "gt_index": match.index[0],
                    "ui_index": index,
                })
            


    matches_df = pd.DataFrame(matches)
    matches_df = check_duplicate_features(matches_df)
    sig_df = matches_df[matches_df["p_value"]<0.05]

    print(f"\n{len(matches_df)} matches found of which {len(sig_df)} are significant features")
    # print(sig_df.head())

    if not os.path.exists(f'../runs/{run_name}/analysis'):
        os.mkdir(f'../runs/{run_name}/analysis')

    # matches_df.to_csv(f'../runs/{run_name}/analysis/matches_{end_name}.csv')
    # sig_df.to_csv(f'../runs/{run_name}/analysis/sig_matches_{end_name}.csv')


In [6]:
# KEGG matches
def run_KEGG_matches():
    matches = []

    for index, row in ui_cpd.iterrows():
        KEGG_id = row['KEGG']

        if pd.notna(KEGG_id):     
            eKEGG_id = str(f"KEGG:{int(KEGG_id)}")
            match = gt[gt['database_identifier'].str.fullmatch(eKEGG_id, case=False, na=False)]
            
            if not match.empty:
                matches.append({
                    "compound_name": row['compound_name'],
                    "gt_compound_name": match['metabolite_identification'].iloc[0],
                    "ui_retention_time": row['retention_time'],
                    "gt_retention_time": match['retention_time'].iloc[0],
                    "rt_difference": abs(row['retention_time'] - match['retention_time'].iloc[0]),
                    "p_value": row['p_value'],
                    "compoundID_from_user": row['CompoundID_from_user'],
                    "gt_KEGG": match['database_identifier'].iloc[0],
                    "ui_KEGG": str(int(KEGG_id)),
                    "gt_index": match.index[0],
                    "ui_index": index,
                })
            

    matches_df = pd.DataFrame(matches)
    matches_df.head()
    matches_df = check_duplicate_features(matches_df)
    sig_df = matches_df[matches_df["p_value"]<0.05]

    print(f"\n{len(matches_df)} matches found of which {len(sig_df)} are significant features")
    # print(sig_df.head())

    if not os.path.exists(f'../runs/{run_name}/analysis'):
        os.mkdir(f'../runs/{run_name}/analysis')

    # matches_df.to_csv(f'../runs/{run_name}/analysis/matches_{end_name}_KEGG2.csv')
    # sig_df.to_csv(f'../runs/{run_name}/analysis/sig_matches_{end_name}_KEGG2.csv')


In [ ]:
run_chebi_matches()